In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import *
import re
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
full = pd.read_csv('final_movies.csv').dropna()

In [3]:
selections = full.drop(columns=['Unnamed: 0','movieId','imdbID','Title','Year','Ratings','Released','Runtime','Plot','Poster','imdbVotes'])

In [ ]:
full = pd.read_csv('clustered_movies.csv')

In [118]:
searched = 'The Hangover'
value = full[full['Title']==searched].index.values[0]

In [119]:
new_full = full[full['clusters'] == full['clusters'][value]].reset_index()
plots_arr = new_full['Plot'].to_numpy()
plots_l = list(plots_arr)

In [120]:
corpus = []
for i in range(0, len(plots_l)):
    text = re.sub('[^a-zA-Z]', ' ', plots_l[i])
    text = text.lower()
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    text=re.sub("(\\d|\\W)+"," ",text)
    corpus.append(text)

In [121]:
vectorizer = TfidfVectorizer(max_features=100, stop_words='english')
X = vectorizer.fit_transform(corpus)

In [122]:
my_tags = vectorizer.get_feature_names()

In [123]:
my_tags

['american',
 'best',
 'big',
 'boss',
 'boy',
 'brother',
 'business',
 'city',
 'college',
 'comedian',
 'comedy',
 'comes',
 'company',
 'country',
 'couple',
 'daughter',
 'dave',
 'day',
 'decide',
 'decides',
 'different',
 'family',
 'father',
 'film',
 'finds',
 'french',
 'friend',
 'friends',
 'gets',
 'girl',
 'girlfriend',
 'girls',
 'goes',
 'good',
 'group',
 'help',
 'high',
 'home',
 'house',
 'husband',
 'italian',
 'job',
 'just',
 'life',
 'like',
 'little',
 'live',
 'lives',
 'living',
 'local',
 'long',
 'love',
 'make',
 'man',
 'married',
 'meet',
 'meets',
 'men',
 'money',
 'mother',
 'movie',
 'new',
 'night',
 'old',
 'order',
 'party',
 'people',
 'police',
 'rich',
 'run',
 'save',
 'school',
 'series',
 'set',
 'small',
 'son',
 'special',
 'stand',
 'story',
 'takes',
 'team',
 'things',
 'time',
 'town',
 'tries',
 'try',
 'trying',
 'village',
 'war',
 'way',
 'wife',
 'woman',
 'women',
 'work',
 'working',
 'world',
 'writer',
 'year',
 'years',
 'yo

In [99]:
my_tag_matrix = pd.DataFrame(0, index=np.arange(len(plots_l)), columns=my_tags)

In [100]:
for i in range(0,len(my_tag_matrix)):
    for j in range(0,len(my_tags)):        
        if my_tags[j] in new_full['Plot'].iloc[i]:
            my_tag_matrix.iloc[i][my_tags.index(my_tags[j])] = 1

In [101]:
my_tag_matrix['imdbID'] = new_full['imdbID']

In [102]:
new_value = new_full[new_full['Title']==searched].index.values[0]

In [103]:
new_selections = my_tag_matrix.drop(columns = ['imdbID'])

In [104]:
thing = new_selections.iloc[new_value]

In [105]:
knn = NearestNeighbors(n_neighbors=4)

In [106]:
knn.fit(new_selections)
arr = knn.kneighbors([thing], return_distance = False)

In [107]:
arr[0][1:4]

array([ 72, 220, 232])

In [108]:
arr

array([[761,  72, 220, 232]])

In [109]:
results = []
for i in arr[0][1:4]:
    url = f'https://imdb.com/title/{new_full["imdbID"].iloc[i]}'
    plot= new_full['Plot'].iloc[i]
    title = new_full['Title'].iloc[i]
    poster = new_full['Poster'].iloc[i]
    temp_dict = {
        'title':title,
        'url':url,
        'plot':plot,
        'poster':poster
    }
    results.append(temp_dict)

In [110]:
results

[{'title': 'Pretty in Pink',
  'url': 'https://imdb.com/title/tt0091790',
  'plot': 'A poor girl must choose between the affections of dating her childhood sweetheart or a rich but sensitive playboy.',
  'poster': 'https://m.media-amazon.com/images/M/MV5BOGE3ZmVjMzMtMDcxNi00YjM5LTg3NjMtZWMxYjlhNTk2MmZjXkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_SX300.jpg'},
 {'title': 'The Man Without a Past',
  'url': 'https://imdb.com/title/tt0311519',
  'plot': 'M arrives in Helsinki only to be viciously attacked by thugs and pronounced dead by medics. He revives but with no memory of his past or his identity. He rebuilds his life from scratch, but the past inevitably catches up with him.',
  'poster': 'https://m.media-amazon.com/images/M/MV5BMTAwNTA0NzEtY2U4MS00MDFlLWJjYWEtZTdlMzRhZTJjMWJlXkEyXkFqcGdeQXVyMDI2NDg0NQ@@._V1_SX300.jpg'},
 {'title': 'Under the Tuscan Sun',
  'url': 'https://imdb.com/title/tt0328589',
  'plot': 'A writer impulsively buys a villa in Tuscany in order to change her life.',
  'poster':